### **Import Libraries**

In [1]:
import pandas as pd
import numpy as np
import random
import os
import torch
import pickle
import tensorflow as tf
from accelerate import Accelerator
from torch.optim import AdamW
from evaluate import load
from typing import Optional, Iterable
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from transformers import PretrainedConfig
from transformers import InformerConfig, InformerForPrediction
from gluonts.dataset.common import ListDataset
from gluonts.itertools import Cached, Cyclic
from gluonts.dataset.loader import as_stacked_batches
from gluonts.transform.sampler import InstanceSampler
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.time_feature import get_lags_for_frequency
from gluonts.time_feature import time_features_from_frequency_str
from gluonts.time_feature import TimeFeature
from gluonts.dataset.field_names import FieldName
from gluonts.transform import (AddAgeFeature,AddObservedValuesIndicator,AddTimeFeatures,AsNumpyArray,Chain,ExpectedNumInstanceSampler,
                               InstanceSplitter,RemoveFields,SelectFields,SetField,TestSplitSampler,Transformation,ValidationSplitSampler,VstackFeatures,RenameFields)
from Functions import *
tf.keras.utils.set_random_seed(0)

data_path = "M:/Dissertation/Data/"
test_data_path = "M:/Dissertation/Return_Prediction/Deep_Learning/Results/"
results_path = "M:/Dissertation/Return_Prediction/Transformer/Results/"

### **Set Informer Params**

In [2]:
# Data Related
freq = '1H'                            # Freq of Data Samples
prediction_length = 1                  # Prediction Horizon
context_length = prediction_length * 7 # Look Back
num_of_variates = 4                    # Number of Time Series (Target Variables)
num_time_features = 4                  # Number of Time Features during Transformation
lags = [1]                             # Number of Lags to Consider

# Model Related
learning_rate = 6e-4
epochs = 25
dropout = 0.3
encoder_layers = 6
decoder_layers = 4
d_model = 32
attention_type = 'full'

### **Extracting Test Portions**  

In [3]:
test_portions = pd.read_csv(test_data_path+"Test_Results.csv")
test_portions["Date"] = pd.to_datetime(test_portions["Date"],format="%Y-%m-%d %H:00:00")
test_portions = pd.concat([test_portions.groupby(by=["Portion"],as_index=False).agg({"Date":"min"}),
                           test_portions.groupby(by=["Portion"]).agg({"Date":"max"})],ignore_index=True,axis=1).rename(columns={0:"Portion",1:"MinDate",2:"MaxDate"})
test_portions

,Portion,MinDate,MaxDate
0,0,2016-06-20 23:00:00,2016-06-24 01:00:00
1,1,2017-11-06 08:00:00,2017-11-09 10:00:00
2,2,2017-09-07 10:00:00,2017-09-12 13:00:00
3,3,2016-07-22 08:00:00,2016-07-27 11:00:00
4,4,2016-10-31 06:00:00,2016-11-03 08:00:00


### **Training the Model and Generating Predictions for Each Test Portion**

In [4]:
# Reading the Data
all_data = pd.read_csv(data_path+"Forex_Data.csv")
all_data["Date"] = pd.to_datetime(all_data["Date"],format="%Y-%m-%d %H:00:00")

# Iterating over the Test Portions
final_results = pd.DataFrame()
for p in list(test_portions.Portion.unique()):
    
    # Slicing the Data as per the Min and Max Dates of Each Test Portion
    min_date = test_portions.loc[test_portions.Portion==p,"MinDate"].iloc[0]
    max_date = test_portions.loc[test_portions.Portion==p,"MaxDate"].iloc[0]
    data = all_data.loc[(all_data.Date>=min_date-pd.DateOffset(years=3))&(all_data.Date<=max_date)].reset_index(drop=True)

    for col in ['EUR/USD_T','EUR/GBP_T','GBP/USD_T','XAU/USD_T']:
        data[col] = data[col.split('_')[0]+'_R']

    for col in ['EUR/USD_R','EUR/GBP_R','GBP/USD_R','XAU/USD_R']:
        data[col] = data[col].shift(1)
        
    data = data.dropna(subset=['EUR/USD_R','EUR/GBP_R','GBP/USD_R','XAU/USD_R'])
    data = data.sort_values(by=["Date"]).reset_index(drop=True)
    
    # Defining the Length of Test Portion
    data["Portion"] = "Train"
    data.loc[(data.Date>=min_date)&(data.Date<=max_date),"Portion"] = "Test"
    first_test_index = data.loc[data['Portion']=="Test"].index[0]
    data.loc[first_test_index-(context_length+max(lags)-1):,["Portion"]] = "Test"
    
    # Transforming the Date Column as per Informer Requirements
    data["Orig_Date"] = data["Date"]
    data["Date"] = pd.date_range(data["Date"].iloc[0],data["Date"].iloc[0]+pd.DateOffset(hours=len(data)-1),freq='1H')
    data = data[["Orig_Date","Date","GBP/USD_T","EUR/GBP_T","EUR/USD_T","XAU/USD_T","GBP/USD","EUR/USD","EUR/GBP","XAU/USD","Portion"]]
    data = data.rename(columns={"Date":"start"})
    
    # Train, Validation and Test Split
    train_data,valid_data,test_data = data_split(df=data,train_size=0.95)

    # Making Copies of Dataframes for later use
    orig_valid = valid_data.copy()
    orig_valid = orig_valid.rename(columns={"Orig_Date":"Date"})
    orig_valid = orig_valid.drop('start',axis=1)

    orig_test = test_data.copy()
    orig_test = orig_test.rename(columns={"Orig_Date":"Date"})
    orig_test = orig_test.drop('start',axis=1)
    
    train_data = train_data.drop(["Orig_Date","GBP/USD","EUR/USD","EUR/GBP","XAU/USD"],axis=1).reset_index(drop=True)
    valid_data = valid_data.drop(["Orig_Date","GBP/USD","EUR/USD","EUR/GBP","XAU/USD"],axis=1).reset_index(drop=True)
    test_data = test_data.drop(["Orig_Date","GBP/USD","EUR/USD","EUR/GBP","XAU/USD"],axis=1).reset_index(drop=True)
    
    # Defining the Informer Model
    config = InformerConfig(input_size = num_of_variates,
                            prediction_length = prediction_length,
                            context_length = context_length,
                            lags_sequence = lags,
                            num_time_features = num_time_features,   
                            dropout=dropout,
                            encoder_layers=encoder_layers,
                            decoder_layers=decoder_layers,
                            d_model=d_model,
                            attention_type=attention_type)
    model = InformerForPrediction(config)
    
    # Transforming Train Data using DataLoader 
    train_dataloader = create_dataloader(train_data,num_of_variates,freq,config,'train')

    # Model Training
    accelerator = Accelerator()
    device = accelerator.device
    optimizer = AdamW(model.parameters(),lr=learning_rate,betas=(0.9,0.95),weight_decay=1e-1)
    model = train_model(train_dataloader,model,device,optimizer,accelerator,epochs,config)

    # Generating Predictions on the Validation Set
    valid_forecasts = generate_forecasts(valid_data,model,num_of_variates,context_length,lags,freq,config,device)
    valid_actual = orig_valid.iloc[(context_length+max(lags)-1):].reset_index(drop=True)
    valid_pred = pd.DataFrame(data=valid_forecasts,columns=['GBP/USD_P','EUR/GBP_P','EUR/USD_P','XAU/USD_P'])
    valid_comb = pd.concat([valid_actual,valid_pred],axis=1)

    # Generating Predictions on the Test Set
    test_forecasts = generate_forecasts(test_data,model,num_of_variates,context_length,lags,freq,config,device)
    test_actual = orig_test.iloc[(context_length+max(lags)-1):].reset_index(drop=True)
    test_pred = pd.DataFrame(data=test_forecasts,columns=['GBP/USD_P','EUR/GBP_P','EUR/USD_P','XAU/USD_P'])
    test_comb = pd.concat([test_actual,test_pred],axis=1)
    test_comb["Test_Portion"] = p
    
    # Mean Squared Error Calculation
    valid_mses = []
    test_mses = []
    for col in ['GBP/USD','EUR/GBP','EUR/USD','XAU/USD']:

        test_comb[col+'_PP'] = (test_comb[col+'_P']+1) * test_comb[col]
        test_comb[col+'_PP'] = test_comb[col+'_PP'].shift(1)

        valid_mses.append(mean_squared_error(valid_comb[col+'_T'],valid_comb[col+'_P']))
        test_mses.append(mean_squared_error(test_comb[col+'_T'],test_comb[col+'_P']))

    # Saving Test Results
    final_results = pd.concat([final_results,test_comb])

    print('#' * 25)
    print('### Test Portion', p+1)
    print("### Average Valid MSE = ",np.mean(valid_mses))
    print("### Average Test MSE = ",np.mean(test_mses))
    print('#' * 25)

final_results.to_csv(results_path+"Test_Results.csv",index=False)

#########################
### Test Portion 1
### Average Valid MSE =  9.631020622341163e-07
### Average Test MSE =  1.6203801900123401e-06
#########################
#########################
### Test Portion 2
### Average Valid MSE =  6.148287286297474e-07
### Average Test MSE =  4.738443974832554e-07
#########################
#########################
### Test Portion 3
### Average Valid MSE =  5.965429248337623e-07
### Average Test MSE =  7.958484591976849e-07
#########################
#########################
### Test Portion 4
### Average Valid MSE =  1.49343179248529e-06
### Average Test MSE =  7.94709269580877e-07
#########################
#########################
### Test Portion 5
### Average Valid MSE =  7.779795173200022e-07
### Average Test MSE =  7.509038245457935e-07
#########################
